<a href="https://colab.research.google.com/github/denny-erikson/dalilas-code-invoice/blob/main/Dalila's_code_Invoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y poppler-utils # Instala o Poppler
!apt-get install tesseract-ocr-por # Installs the Portuguese language data for Tesseract
!pip install pytesseract Pillow pdf2image # Keep your existing installations

import pytesseract
from PIL import Image
import re
import os
from pdf2image import convert_from_path # Importa a função para converter PDF em imagens
from datetime import datetime # Importa o módulo datetime para gerar data e hora

# Monta o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the TESSDATA_PREFIX environment variable
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata' # Set the path for tessdata

def extract_info_and_save(file_path, save_dir):
    # Verifica a extensão do arquivo
    file_extension = os.path.splitext(file_path)[1].lower()

    # Variável para armazenar o log de erros
    error_log = []

    if file_extension == '.pdf':
        # Converte PDF para imagens
        images = convert_from_path(file_path)

        # Usa o Tesseract para extrair texto de cada página do PDF
        for page_number, image in enumerate(images, start=1):
            text = pytesseract.image_to_string(image, lang='por')  # Extrai texto de cada página

            # Função para encontrar as informações de "Número da NFS-e" e "Nome/Razão Social"
            def extract_info(text):
                # Expressão regular para capturar o "Número da NFS-e"
                nfs_e_match = re.search(r'Número da NFS-e\s*(\d+)', text)
                nfs_e_number = nfs_e_match.group(1) if nfs_e_match else "Não encontrado"

                # Expressão regular para capturar o "Nome/Razão Social"
                name_match = re.search(r'Nome/Razão Social\s*(.+)', text)
                name = name_match.group(1).strip() if name_match else "Não encontrado"

                return nfs_e_number, name

            # Extrai as informações
            try:
                nfs_e_number, name = extract_info(text)

                # Salva cada página como um PDF separado
                safe_name = re.sub(r'[<>:"/\\|?*]', '', name)  # Remove caracteres inválidos para nomes de arquivos
                safe_nfs_e_number = nfs_e_number if nfs_e_number != "Não encontrado" else "0000"  # Valor padrão
                formatted_file_name = f"NFSe-{safe_nfs_e_number}-{safe_name}-Page-{page_number}.pdf"

                # Define o caminho completo para salvar
                save_path = os.path.join(save_dir, formatted_file_name)

                # Salva a imagem da página como PDF
                image.save(save_path, "PDF")  # Salva como PDF
                print(f"Arquivo PDF da página {page_number} salvo como: {save_path}")

            except Exception as e:
                error_log.append(f"Falha na leitura/salvamento da página {page_number}: {str(e)}")

    else:
        # Carrega a imagem
        image = Image.open(file_path)

        # Usa o Tesseract para extrair texto da imagem
        text = pytesseract.image_to_string(image, lang='por')  # Define o idioma como português

        # Função para encontrar as informações de "Número da NFS-e" e "Nome/Razão Social"
        def extract_info(text):
            # Expressão regular para capturar o "Número da NFS-e"
            nfs_e_match = re.search(r'Número da NFS-e\s*(\d+)', text)
            nfs_e_number = nfs_e_match.group(1) if nfs_e_match else "Não encontrado"

            # Expressão regular para capturar o "Nome/Razão Social"
            name_match = re.search(r'Nome/Razão Social\s*(.+)', text)
            name = name_match.group(1).strip() if name_match else "Não encontrado"

            return nfs_e_number, name

        # Extrai as informações
        try:
            nfs_e_number, name = extract_info(text)

            # Exibe as informações extraídas
            print(f"Número da NFS-e: {nfs_e_number}")
            print(f"Nome/Razão Social: {name}")

            # Limpa os nomes para garantir que sejam seguros para uso em arquivos
            safe_name = re.sub(r'[<>:"/\\|?*]', '', name)  # Remove caracteres inválidos para nomes de arquivos
            safe_nfs_e_number = nfs_e_number if nfs_e_number != "Não encontrado" else "0000"  # Valor padrão

            # Salva a imagem com o nome formatado
            formatted_file_name = f"NFSe-{safe_nfs_e_number}-{safe_name}{file_extension}"

            # Define o caminho completo para salvar
            save_path = os.path.join(save_dir, formatted_file_name)

            image.save(save_path)  # Salva a imagem com o novo nome
            print(f"Arquivo salvo como: {save_path}")

        except Exception as e:
            error_log.append(f"Falha na leitura/salvamento da imagem: {str(e)}")

    # Se houver erros, registra no log
    if error_log:
        log_file_path = os.path.join(save_dir, "error_log.txt")
        with open(log_file_path, "w") as log_file:
            for error in error_log:
                log_file.write(error + "\n")
        print(f"Erros registrados em: {log_file_path}")

# Carregar a imagem ou PDF
from google.colab import files
uploaded = files.upload()

# Define a pasta onde os arquivos serão salvos (exemplo: '/content/drive/MyDrive/NFSe')
base_directory = '/content/drive/MyDrive/NFSe'
# Cria um nome de pasta baseado na data e hora atual
current_time = datetime.now().strftime("%d-%m-%Y_%H-%M")
save_directory = os.path.join(base_directory, current_time)
os.makedirs(save_directory, exist_ok=True)  # Cria a pasta se não existir

# Executar a função
file_path = list(uploaded.keys())[0]  # Obtém o nome do arquivo carregado
extract_info_and_save(file_path, save_directory)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-por is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving ilovepdf_merged.pdf to ilovepdf_merged (1).pdf
Arquivo PDF da página 1 salvo como: /content/drive/MyDrive/NFSe/11-10-2024_13-07/NFSe-1924-SANTA CASA DE MISERICRODIA DE SÃO BENTO DO SAPUCAÍ-Page-1.pdf
Arquivo PDF da página 2 salvo como: /content/drive/MyDrive/NFSe/11-10-2024_13-07/NFSe-1924-SANTA CASA DE MISERICRODIA DE SÃO BENTO DO SAPUCAÍ-Page-2.pdf
Arquivo PDF da página 3 salvo como: /content/drive/MyDrive/NFSe/11-10-2024_13-07/NFSe-1924-SANTA CASA DE MISERICRODIA DE SÃO BENTO DO SAPUCAÍ-Page-3.pdf
